# Signal to noise analysis

Does RLS really have better signal to noise (sensitivity) for detecting QLLI compared to Qd?

To answer this question, let's consider the resolution limit of either measurement when considering a certain equipment spec

In [1]:
import numpy as np
import pandas as pd

In [2]:
# Define some parameters

# Percent of full scale range, voltage
voltage_precision_perc = 0.02

# Voltage full scale range in Volts
voltage_fsr = 5

# Percent of full scale range, current
current_precision_perc = 0.02 

# Current full scale range in Ameres
current_fsr = 5 

# Calculate the errors
current_err = current_fsr * current_precision_perc / 100
voltage_err = voltage_fsr * voltage_precision_perc / 100

# Linearization of sensitivity of Q_LLI to RLS and Qd

# mohm/mah
rls_per_qlli_vec = np.array([14, 9, 5, 1])/40
soc_vec = [2, 5, 7, 10]

# mah/mah
qd_per_qlli = 36/40 # Assuming no LAM

In [3]:
print(current_err)
print(voltage_err)

0.001
0.001


In [4]:
# Resistance calculation
voltage_meas = 0.1
current_meas = 2.37

resistance_lo = (voltage_meas - voltage_err) / (current_meas + current_err)
resistance_hi = (voltage_meas + voltage_err) / (current_meas - current_err)

resistance_resolution = resistance_hi - resistance_lo

In [5]:
# Capacity calculation
current_meas = 2.37 / 10 # C/10 discharge
duration_hours = 10

# Assume we do constant current charging.
# Ignore CV contributions
capacity_lo = (current_meas - current_err) * duration_hours
capacity_hi = (current_meas + current_err) * duration_hours

capacity_resolution = capacity_hi - capacity_lo

In [6]:
resistance_resolution_mohms = resistance_resolution * 1000
capacity_resolution_mah = capacity_resolution * 1000

In [7]:
resistance_resolution_mohms

0.8794888425090047

In [8]:
capacity_resolution_mah

20.000000000000018

In [9]:
for rls_per_qlli, soc in zip(rls_per_qlli_vec, soc_vec):
    qlli_resolution_by_rls_mah = resistance_resolution_mohms / rls_per_qlli
    print(f'SOC = {soc}% : {qlli_resolution_by_rls_mah:.3f} mAh')



SOC = 2% : 2.513 mAh
SOC = 5% : 3.909 mAh
SOC = 7% : 7.036 mAh
SOC = 10% : 35.180 mAh


In [10]:
qlli_resolution_by_qd_mah = capacity_resolution_mah / qd_per_qlli
qlli_resolution_by_qd_mah

22.222222222222243